**Hunter Mitchell**

**Movie Popularity Prediction Project**

**September 22nd, 2020**

# Intro

***What makes a movie popular?***

This project attempts to answer that question by examining various factors of films including budget, run time, genre, language and rating!

We will utilize Machine Learning and Data Science fundamentals to create a model that will predict a movie's popularity. This could be useful for film companies to determine what factors into a movie's popularity, or for other content creators to better understand their audience.

The data I am using is obtained from a Kaggle dataset found [here](https://www.kaggle.com/tmdb/tmdb-movie-metadata). I encourage anyone to explore the data for themselves and predict other potentially useful imformation (revenue, rating, etc.) 

We must also verify four main assumptions to get valid results from Linear Regression. These are:

1.   Linearity between features and target
2.   Multivariate normality
3.   Little multicollinearity
4.   Homoscedasticity 

We will examine each of these before implementing a Linear Regression model. Now let's get started!

# Getting to know our data

Let's begin by importing some necessary libraries so that we can explore our data!

In [ ]:
import pandas as pd # for dataframes
import numpy as np # for arrays & math functions

%matplotlib inline
import matplotlib.pyplot as plt # for plotting

import warnings
warnings.filterwarnings('ignore') # ignoring any warnings

In [ ]:
PATH = '../input/tmdb-movie-metadata/tmdb_5000_movies.csv'

movies_df = pd.read_csv(PATH) # load data into a pandas dataframe

Let's take a look at a couple instances to see what information we have

In [ ]:
SEED = 2020 # for reproducability

movies_df.sample(3,random_state=SEED)

We can see already that there is some missing data that we will need to take care of. There are also entries of zero that could be false.

According to the dataset, popularity is measured as the cumulative number of star ratings. It is also unknown whether the budget and revenue are in USD or some other currency. 

Let's look at all of our different columns.


In [ ]:
movies_df.info()

As you can see, we have 19 columns to work with. Many of these will not correlate to popularity, though.

Overall, we don't have many null values. There are a lot in the homepage column, but that shouldn't be correlated to popularity, so we can safely ignore it as a feature. Our dataset contains 4,803 instances: not a ton, but it should be plenty for a regression model.

Now let's check out our target variable 'popularity'.

In [ ]:
movies_df['popularity'].describe()

It looks like we may have a few outliers present. To see if those are valid instances or not, let's look at the corresponding rows.

In [ ]:
movies_df.sort_values(by='popularity',ascending=False)[:5]

Wow! I rememeber the Minions movie being popular but not *that* popular! The other top movies seem valid as well - I was working at a movie theater when Deadpool came out, and it was massively popular!

Although these are outliers, they are still valid instances, and thus we will not modify or drop any of them.

Now let's look at some of the lowest popularities.

In [ ]:
movies_df.sort_values(by='popularity',ascending=True)[:5]

This provides insight into what type of instances we may need to drop - two of these have a runtime of 0! We can also see some missing or suspicious values that we will need to look at. Furthermore, I'm a bit worried about instances when the vote count is very low, as this will make vote average not representative. Let's look at some of the vote average extremes.

In [ ]:
movies_df.sort_values(by='vote_average', ascending = False)[:5]

Sure enough, the movies with the best vote averages were only reviewed one or two times! We will have to cut off a certain threshold of vote counts to fix this.

Now let's take a look at the different languages we have.

In [ ]:
movies_df['original_language'].value_counts()

In [ ]:
### Pie Chart

labels = np.array(['English','Other'])
sizes = np.array([4505, sum(movies_df['original_language'].value_counts()) - 4505])

plt.figure(figsize=(8,9))

plt.pie(sizes, labels=labels, autopct='%1.1f%%', explode=[0,0.08], startangle=90)
plt.title('Original Languages of Movies', fontdict={'fontsize': 14})
plt.axis('equal')

93.8% of the movies are in english. There are a few different ways we could encode this column. One method would be to one-hot encode by creating a binary column for each different language. This would add a ton of new features to the data, with some only having 1 or 2 positive instances. Another way we could encode this is by converting it to just english or not english. This would only add one column and likely contribute to popularity still.

# Cleaning

Let's create our features dataframe and clean it up a bit. I am choosing all the features that I believe impact a movies popularity. I am also leaving out revenue as a feature, as this is something revealed a while after a movie comes out.

In [ ]:
features_df = movies_df[['budget','genres','original_language','runtime','vote_average','vote_count']]
labels_df = movies_df['popularity']

First let's drop any rows with null values

In [ ]:
features_df = features_df.dropna()

Now let's drop instances with very few vote counts. I decided to have 10 as the cutoff - if 10 or more people voted, it will likely have a fairly accurate vote average. 

In [ ]:
features_df = features_df[features_df['vote_count'] >= 10]

Now let's drop any values with a runtime of zero

In [ ]:
features_df = features_df[features_df['runtime'] != 0.0]

And lastly, we have to set the labels dataframe to only include the rows of the new features dataframe

In [ ]:
labels_df = labels_df[features_df.index]

Now let's get some insight into our cleaned features

In [ ]:
features_df.describe()

We can already see that the average vote column has no zeros and no 10s, which is good. Furthermore, the shortest runtime is 25 minutes and the longest is 338 minutes, which seem accurate.

There seem to be some films that have a budget of 0. We will keep these as valid because there are plenty of low-budget films (think Blair Witch Project)

# Exploratory Data Analysis

Now it is time to split our data into training and testing data. This is crucial, as we do not want to notice any overall patterns before our models run. We will only use the training data from now on and use our testing data to test our final models.

In [ ]:
from sklearn.model_selection import train_test_split

# split data into 80% training and 20% testing
x_train, x_test, y_train, y_test = train_test_split(features_df, labels_df, test_size=0.2, random_state = SEED)

Let's look at our target variable distribution by plotting a histogram.

In [ ]:
# Plot histogram
plt.figure(figsize=(10,8))
plt.hist(y_train.values,bins=50)
plt.title('Movies popularity histogram')
plt.xlabel('Popularity')
plt.ylabel('# of movies')
plt.show()

The data is heavily skewed left. The skew method will return how asymmetric the data is (with zero being completely symmetric)

In [ ]:
y_train.skew()

Linear regression should have all variables be approximately normal, so let's fix this. There are a couple ways we could deal with skewed distributions. These include applying logarithms, square roots, or using the box cox method. I am going to use the log method here. Note that we have to do this to our testing data too. 

In [ ]:
y_train = np.log(y_train)
y_test = np.log(y_test)

In [ ]:
# Plot histogram
plt.figure(figsize=(10,8))
plt.hist(y_train.values,bins=50)
plt.title('Movies popularity histogram')
plt.xlabel('Popularity')
plt.ylabel('# of movies')
plt.show()

That looks much better, but let's verify by checking the skew

In [ ]:
print('Popularity skew:', y_train.skew())

Great! Now let's look at the numerical features

In [ ]:
### Plot histograms
plt.rcParams['figure.figsize'] = 12, 12
fig, axs = plt.subplots(2,2)
fig.suptitle('Numerical Feature Histograms',y=0.95,fontsize=16)

axs[0,1].hist(x_train['budget'].values,bins=30,color='salmon')
axs[0,1].set_title('Budget')
axs[0,1].set(xlabel='US dollars')
axs[0,0].hist(x_train['runtime'].values,bins=30,color='salmon')
axs[0,0].set_title('Runtime (min)')
axs[0,0].set(xlabel='Minutes')
axs[1,0].hist(x_train['vote_average'].values,bins=30,color='salmon')
axs[1,0].set_title('Vote Average')
axs[1,1].hist(x_train['vote_count'].values,bins=30,color='salmon')
axs[1,1].set_title('Vote Count')
plt.show()

Both runtime and vote average seem fairly normal, but vote count and budget are definitely skewed.

In [ ]:
print('Vote count skew:', x_train['vote_count'].skew())
print('Vote average skew:', x_train['vote_average'].skew())
print('Runtime skew:', x_train['runtime'].skew())
print('Budget skew:', x_train['budget'].skew())

Let's normalize the budget and vote count variables. We will do another log transformation to vote count and do a square root transformation to budget. This is because budget contains zero values and taking the log of these would give us undefined values. 

In [ ]:
x_train['vote_count'] = np.log(x_train['vote_count'].values)
x_train['budget'] = np.sqrt(x_train['budget'].values)

x_test['vote_count'] = np.log(x_test['vote_count'].values)
x_test['budget'] = np.sqrt(x_test['budget'].values)

Now let's look at the distributions and skew values

In [ ]:
### Histograms

plt.rcParams['figure.figsize'] = 12, 12
fig, axs = plt.subplots(2,2)
fig.suptitle('Numerical Feature Histograms',y=0.95,fontsize=16)

axs[0,1].hist(x_train['budget'].values,bins=30,color='salmon')
axs[0,1].set_title('Budget')
axs[0,1].set(xlabel='US dollars')
axs[0,0].hist(x_train['runtime'].values,bins=30,color='salmon')
axs[0,0].set_title('Runtime (min)')
axs[0,0].set(xlabel='Minutes')
axs[1,0].hist(x_train['vote_average'].values,bins=30,color='salmon')
axs[1,0].set_title('Vote Average')
axs[1,1].hist(x_train['vote_count'].values,bins=30,color='salmon')
axs[1,1].set_title('Vote Count')
plt.show()

In [ ]:
print(x_train['vote_count'].skew())
print(x_train['vote_average'].skew())
print(x_train['runtime'].skew())
print(x_train['budget'].skew())

We could work to normalize these more, but for the sake of this project, we will keep it how it is. Now that we have made the distributions more normal, let's look at the correlations between our variables.

In [ ]:
# Scatterplots

plt.figure(figsize=(12,12))

fig, axs = plt.subplots(2,2)
fig.suptitle('Correlation to target',y=0.95,fontsize=16)
axs[0,1].scatter(x_train['vote_average'].values, y_train.values,color='green')
axs[0,1].set(xlabel='Vote Average',ylabel='Populariy')
axs[0,0].scatter(x_train['budget'].values, y_train.values,color='green')
axs[0,0].set(xlabel='Budget (US dollars)',ylabel='Populariy')
axs[1,0].scatter(x_train['vote_count'].values, y_train.values,color='green')
axs[1,0].set(xlabel='Vote Count',ylabel='Populariy')
axs[1,1].scatter(x_train['runtime'].values, y_train.values,color='green')
axs[1,1].set(xlabel='Runtime (min)',ylabel='Populariy')
plt.show()

Vote count definitely looks the most correlated to popularity, but let's check their actual correlation coefficients.

In [ ]:
corr_matrix = pd.concat([x_train,y_train],axis=1).corr()

corr_matrix['popularity'].sort_values(ascending=False)

Vote count is indeed the most correlated. The others are still correlated and we can now check off the linearity assumption of Linear Regression. We can also check off our homoscedasticity assumption as the residuals are approximately equal throughout each scatterplot. If they were not equal, we would see a strong cone shape. Now let's look at how our features correlate to each other.

In [ ]:
from pandas.plotting import scatter_matrix

scatter_matrix(x_train[['budget','runtime','vote_count','vote_average']], figsize=(12,12))

We can see that there may be a bit of correlation between our independent variables. Let's look at the correlation coefficients.

In [ ]:
x_train.corr()

While there is a bit more correlation between some independent variables than we would like, we will choose to ignore it for the scope of this project. Therefore, we now have considered all four assumptions and we can move on to getting the data ready for our models!

# Feature Engineering

We still have categorical features that we need to encode. 

For language, we will make a binary variable with 1 for english and 0 for non-english. 

In [ ]:
### Encode language 

x_train['Language'] = x_train['original_language'].apply(lambda x: 1 if 'en' == x else 0)
x_test['Language'] = x_test['original_language'].apply(lambda x: 1 if 'en' == x else 0)

For genres, we will choose to one-hot encode them. Normally, we would use a OneHotEncoder class here, however it will not work with the genre objects we have. This is because every genre entry is a dict with all of the genres they contain, so a OneHotEncoder would make a unique column for each combination. We will have to do it manually.

In [ ]:
genre_list = ['Action', 'Adventure', 'Fantasy', 'Science Fiction', 'Crime', 'Drama', 'Thriller', 'Animation',
 'Family', 'Western', 'Comedy', 'Romance', 'Horror', 'Mystery', 'History', 'War', 'Music']

In [ ]:
# make column for each genre and encode it

for genre in genre_list:
  x_train[genre] = x_train['genres'].apply(lambda x: 1 if genre in x else 0)
  x_test[genre] = x_test['genres'].apply(lambda x: 1 if genre in x else 0)

In [ ]:
x_train.describe()

In [ ]:
corr_matrix = pd.concat([x_train,y_train],axis=1).corr()

corr_matrix['popularity'].sort_values(ascending=False)

These results are interesting in themselves! It looks like the adventure and action genres correlate most positively to popularity, and comedy, romance, and drama correlate most negatively. There are a few genres that do not seem to correlate at all, but we will choose to keep them as features.

We also have to remember to drop our original categorical columns! 

In [ ]:
x_train = x_train.drop(columns=['original_language','genres']) # drop encoded columns
x_test = x_test.drop(columns=['original_language','genres']) # drop encoded columns

Now let's scale our features. Machine Learning models tend to perform better when scaled.

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = MinMaxScaler()

x_train_scaled = pd.DataFrame(scaler.fit_transform(x_train),index=x_train.index, columns=x_train.columns)
x_test_scaled = pd.DataFrame(scaler.transform(x_test),index=x_test.index, columns=x_test.columns)

In [ ]:
# Verify

x_train_scaled.head()

Perfect! Now we are finally ready to train some models!

# Model Selection

With the data all ready to go, it is finally time to train some models! Let's define a few lists to score our metrics, and a function to print them. The cross_val_score will use cross validation to score the model predictions on all parts of our training data. The main metric we will be looking at is Root Mean Squared Error. This is a common metrics for regression tasks. 

In [ ]:
from sklearn.model_selection import cross_val_score

rmse_list = []
std_list = []

def get_score(model):
  cv_score = cross_val_score(model, x_train_scaled, y_train, scoring = "neg_mean_squared_error", cv = 8)
  rmse = np.sqrt(-cv_score)
  print('Cross-Validation Root Mean Squared Error:', rmse)
  print('Average Root Mean Squared Error:', round(np.mean(rmse), 5))
  rmse_list.append(round(np.mean(rmse), 5))
  print('Standard deviation:', round(rmse.std(), 5))
  std_list.append(round(rmse.std(), 5))

## Regression

We will begin with the classic linear regression models. Once again, the results and validity of these depend on the four assumptions that we already discussed. 



In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [ ]:
### Linear Regression

model_1 = LinearRegression()

model_1.fit(x_train_scaled,y_train)

get_score(model_1)

Ridge regression adds a regularization term to the cost function which helps prevent overfitting. The term added is the sum of the square of the coefficients.

In [ ]:
### Ridge Regression

model_2 = Ridge(random_state=SEED)

model_2.fit(x_train_scaled,y_train)

get_score(model_2)

Lasso regression also adds a regularization term to the cost function, but this time it is the sum of the magnitudes of the coefficients.

In [ ]:
### Lasso Regression

model_3 = Lasso(random_state=SEED)

model_3.fit(x_train_scaled,y_train)

get_score(model_3)

## Random Forest

Random Forest is a simple yet powerful machine learning model. It uses many decision trees to make it's final decision. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model_4 = RandomForestRegressor(random_state=SEED)

model_4.fit(x_train_scaled,y_train)

get_score(model_4)

## Support Vector Regressor

Support Vector Machines are typically used for classification tasks, but they can be used for regression as well.

In [ ]:
from sklearn.svm import SVR

In [ ]:
model_5 = SVR()

model_5.fit(x_train_scaled,y_train)

get_score(model_5)

## XGBoost

XGBoost uses gradient boosted trees to make it's decision. They often outperform most models when the hyperparameters are tuned correctly.

In [ ]:
from xgboost import XGBRegressor

In [ ]:
model_6 = XGBRegressor(random_state=SEED,verbose=0,objective='reg:squarederror')

model_6.fit(x_train_scaled,y_train)

get_score(model_6)

## Neural Network

Neural Networks have been around for a while, but have recently taken over all areas of machine learning with architecture advancements and better processing units.  

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as L
from keras.wrappers.scikit_learn import KerasRegressor

I played with the architecture a bit (number of layers, dropout, neurons per layer, etc.) and this seemed to yield the best results.

In [ ]:
def get_tf_model():
    model = tf.keras.Sequential([
        L.Input(shape=(x_train_scaled.shape[1])),
        L.Dense(250, activation='relu'),
        L.BatchNormalization(),
        L.Dense(200, activation='relu'),
        L.BatchNormalization(),
        L.Dense(200, activation='relu'),
        L.BatchNormalization(),
        L.Dense(1)
    ])

    model.compile(
        optimizer='adam',
        loss = 'mse',
        metrics=['accuracy','mse']
    )
    
    return model

In [ ]:
get_tf_model().summary()

In [ ]:
model_7 = KerasRegressor(build_fn = get_tf_model, epochs = 10, verbose = 0, batch_size = 100)
model_7.fit(x_train_scaled,y_train.values)

In [ ]:
get_score(model_7)

## Cross-Validation Scores

Let's see how the models compare

In [ ]:
# For creating tables that render in Github
!pip install --upgrade plotly
!pip install -U kaleido

In [ ]:
import plotly.graph_objects as go

# Create table

models_list = ['Linear Regression','Ridge Regression','Lasso Regression','Random Forest', 
               'Support Vector Regressor','XGBoost', 'Neural Network']

fig = go.Figure(data=[go.Table(header=dict(values=['Model', 'RMSE', 'Standard Deviation']),
                 cells=dict(values=[models_list, rmse_list, std_list]))
                     ])

fig.update_layout(
    title={
        'text': "Starting Model Cross Validation Scores",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show("png")

Regular Linear Regression and Ridge Regression perform the best. Lasso Regression definitely performs worse - probably because the hyperparameters aren't tuned. XGBoost, Random Forest, and Support Vector Regressor also perform well. Neural Networks kind of let us down, but if we played with the structure more, I'm sure we could make it better.

# Hyperparameter Tuning

Let's try to improve some of these scores. Kaggle competitors may spend days or even weeks figuring out the best hyperparameters for their models as well as which models to ensemble. 


We will just pick a few to improve by using grid search. Grid search reveals which hyperparameter combinations provide the best results by trying many different combinations from what you give it.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Enter model and parameter options and returns best model
def grid_search(model,params):
  search = GridSearchCV(model, params, cv=5, scoring='neg_mean_squared_error')
  search.fit(x_train_scaled,y_train)
  return search.best_estimator_

### Lasso

Lasso performed the worst, so let's try to improve it by first looking at what parameters there are.

In [ ]:
model_3.get_params()

We'll pick some options, and apply gird search

In [ ]:
param_grid = [
              {'alpha': [0.1,0.05,0.01,0.005] , 
               "fit_intercept": [True, False], 
               'normalize': [True, False],
               "tol": [0.0005,.0001,0.00005]}
]

model_3_grid = grid_search(model_3,param_grid)

model_3_grid.get_params() # these will be our new parameters

In [ ]:
get_score(model_3_grid)

Wow! Just like that our RMSE improved a lot!

### Support Vector Regressor

Now let's try it for Support Vector Regressor

In [ ]:
# Support Vector Regression

model_5.get_params()

In [ ]:
param_grid = [
              {'kernel': ['linear', 'rbf'],
               'tol': [0.015, 0.01],
               'epsilon': [0.2, 0.15] }
]

model_5_grid = grid_search(model_5,param_grid)

model_5_grid.get_params()

In [ ]:
get_score(model_5_grid)

### XGBoost

And lastly we will use grid search on our XGBoost model

In [ ]:
model_6.get_params()

In [ ]:
param_grid = [
              {'gamma': [10,5],
               'max_depth': [7,5],
               'min_child_weight': [30,20],
               'learning_rate': [0.05,0.01]}
]

model_6_grid = grid_search(model_6,param_grid)

model_6_grid.get_params()

In [ ]:
get_score(model_6_grid)

### Comparison

We could do this all day to zero in on the absolute best hyperparemters, but for the sake of this project, we will stop here. Let's look at the comparisons

In [ ]:
# Create table

models_list = ['Lasso Regression','Support Vector Regressor','XGBoost']

fig = go.Figure(data=[go.Table(header=dict(values=['Model', 'Original RMSE', 'Grid Search RMSE', 
                                                   'Original Standard Deviation', 'Grid Search Standard Deviation']),
                 cells=dict(values=[models_list, [rmse_list[2], rmse_list[4], rmse_list[5]], rmse_list[-3:],
                                    [std_list[2], std_list[4], std_list[5]], std_list[-3:]]))
                     ])

fig.update_layout(
    title={
        'text': "Grid Searched Model Comparisons",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.show("png")

Grid search improved Lasso by a lot, improved Support Vector by a bit, and actually didn't improve XGBoost. This is why it is important to keep trying different combinations of hyperparameter values to see what works. It also improved standard deviation in Lasso and Support Vector, but made XGBoost worse.

# Results

Up to this point, we have only looked at our training data. Now we can try out our models on the test data to see how they perform on data they have never seen. We will first declare some arrays to store our predictions and scores, and make a function to calculate the Root Mean Squared Error. 

It is also very important to note that the popularity predictions we get will be the logarithm of the actual popularity predictions. Hence, to get the real popularity prediction, we must take the exponential. This is because we need to reverse the log function we applied earlier when we normalized our target variable. 

In [ ]:
from sklearn.metrics import mean_squared_error

predictions = []
final_scores = []

def get_results(preds):
  score = np.sqrt(mean_squared_error(preds,y_test.values))
  final_scores.append(round(score,5))

Now we can make test predictions for each model

In [ ]:
### Regression

preds = np.array(model_1.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### Ridge

preds = np.array(model_2.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### Lasso

preds = np.array(model_3.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### Forest

preds = np.array(model_4.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### SVR

preds = np.array(model_5.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### XGBoost

preds = np.array(model_6.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### Neural Network

preds = model_7.predict(x_test_scaled).reshape(len(x_test_scaled))
predictions.append(preds)
get_results(preds)


### Grid searched Lasso

preds = np.array(model_3_grid.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### Grid searched SVR

preds = np.array(model_5_grid.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)


### Grid searched XGBoost

preds = np.array(model_6_grid.predict(x_test_scaled))
predictions.append(preds)
get_results(preds)

And just for fun, let's try to ensemble some of these predictions and see how they do. The first ensemble will be an average of every model we have made up to this point. The second ensemble will be an average of just the three grid searched models. The third ensemble will be an average of our top three models up to this point (Linear Regression, Ridge Regression, Grid searched Support Vector Regressor). 


In [ ]:
# average all model predictions
ensemble_1 = np.mean(predictions,axis=0) 
get_results(ensemble_1)


# average last three model predictions
ensemble_2 = np.mean(predictions[-3:],axis=0) 
get_results(ensemble_2)


# average top three model predictions
ensemble_3 = np.mean([predictions[0], predictions[1],predictions[8]],axis=0) 
get_results(ensemble_3)

Let's see how the final RMSEs compare between all these models

In [ ]:
# Create table

models_list = ['Linear Regression','Ridge Regression','Lasso Regression','Random Forest','Support Vector Regressor',
               'XGBoost','Neural Network','Grid Search Lasso','Grid Search Support Vector Regressor',
               'Grid Search XGBoost','All Model Ensemble','Grid Search Model Ensemble', 'Top 3 Ensemble']

models_ranked_df = pd.DataFrame(data={'model': models_list, 'score': final_scores}).sort_values(by='score')

fig = go.Figure(data=[go.Table(header=dict(values=['Model', 'Final RMSE']),
                 cells=dict(values=[models_ranked_df.model, models_ranked_df.score ]))
                     ])

fig.update_layout(
    title={
        'text': "All Models Ranked",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show("png")

Surprisingly, our regular XGBoost model performs the best out of all models! We could definitely make it better by finding the right hyperparameters through a more refined grid search. Our grid searched models also perform well, with two of our ensembles coming in 3rd and 4th place. Most of these results are similar to how the models performed on our training data. This is good because it shows there is little to no overfitting and underfitting happening. We could also try some other ensembles to find the best combination. Normally ensembles work the best because they average out all of the residuals between the models. 

Speaking of residuals, let's look at a residuals plot from our best model (XGBoost)

In [ ]:
from yellowbrick.regressor import ResidualsPlot

visualizer = ResidualsPlot(model_6, is_fitted=True, train_color='b', test_color='g', size=(1080,720))
visualizer.fit(x_train_scaled, y_train)
visualizer.score(x_test_scaled,y_test)
visualizer.poof() 

This plot gives a lot of useful information. First, our residuals distribution on the right is approximately normal. This is good - if it was skewed, we would not be able to accurately use this information for statistical decisions. Looking at the main section now, we can see that the training and testing residuals are very similar. Our training data seems to be a bit more spread out overall, which we can verify by checking the $R^2$ scores at the top. These measure how well our model fit the data, with 1 being perfect. Since the training score is a bit better than our testing score, there is a bit of overfitting occuring. Ideally, these would about be the same. Our residuals also center around 0 which means that they predict more than the actual value just about as often as they predict less.



XGBoost also let's us look at how important the features were in it's predictions

In [ ]:
# Create table

features_ranked_df = pd.DataFrame(data={'feature': x_test_scaled.columns, 
                                        'importance': model_6.feature_importances_}
                                  ).sort_values(by='importance', ascending = False)

fig = go.Figure(data=[go.Table(header=dict(values=['Feature', 'Importance']),
                 cells=dict(values=[features_ranked_df.feature, [round(x,5) for x in features_ranked_df.importance]]))
                     ])


fig.update_layout(
    title={
        'text': "Features Ranked",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show("png")

It looks like vote count was the most important feature. This is expected as it was the most correlated to popularity. We have a somewhat unexpected next most important feature: the War genre. This was one of the least correlated to popularity, so it is definitely surprising to see it so high up on this list. I would have expected our other numerical features to be higher up on this list as well. There are some features not shown as they are approximately of zero importance. This is not too surprising though - there were very few instances with these features to begin with and they showed very little correlation. 

Any regression model can also provide an $R^2$ score. This is the coefficient of determination and shows how well our model fit the data between -1 and 1. The closer the score is to 1 the better. Let's take a look at how these compare.

In [ ]:
def score(model):
  return round(model.score(x_test_scaled, y_test),5)

# Create table

model_r2_list = [score(model_1), score(model_2), score(model_3), score(model_4), score(model_5),
                 score(model_6), score(model_7), score(model_3_grid), score(model_5_grid), score(model_6_grid)]

r2_ranked_df = pd.DataFrame(data={'model': models_list[:10], 'r2':model_r2_list}
                                  ).sort_values(by='r2', ascending = False)

fig = go.Figure(data=[go.Table(header=dict(values=['Model', 'R^2']),
                 cells=dict(values=[r2_ranked_df.model, r2_ranked_df.r2 ]))
                     ])

fig.update_layout(
    title={
        'text': "R^2 Values",
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.show("png")

These results are not too surprising - they end up very similar to our RMSE table. However, these scores show how much room we still have for improvement. Although 0.84393 is a good score, it still has a bit to go before being a great model. We can also see that our original Lasso Regression model, and our Neural Network fit the data pretty terribly. We were able to improve the Lasso model through grid search, but we will need to experiment quite a bit with Neural Networks to make it more accurate. 

# Conclusion

From the models that we tried, XGBoost worked the best. I am confident that with more hyperparameter tuning and ensembling, we could achieve a much better RMSE and $R^2$ score. There are also many other models and methods we could try that are outside the scope of this project. I would like to work with on this further in the future with more data. I would also like to predict other things such as revenue or ratings, and try different feature combinations. Overall, this project gave a lot of insight into what increases the popularity of movies. I think it could be useful for production companies or independent filmmakers. I also personally learned a lot doing this and look forward to working on more complex projects in the future! 